# PID Control System Analysis Notebook

This notebook provides a flexible workflow for PID controller design and analysis.  
You can iteratively specify the plant, tune the controller, analyze stability, margins, discretize for embedded deployment, and repeat as needed.

## 1. Specify Plant Model

Define your plant transfer function here.  
You can modify the numerator, denominator, and delay as needed.

In [ ]:
import numpy as np
from slicot import tf01md, ab04md, tb05ad
import matplotlib.pyplot as plt

def tf_to_ss(num, den):
    """Convert SISO transfer function to controllable canonical state-space."""
    num = np.atleast_1d(num).astype(float)
    den = np.atleast_1d(den).astype(float)
    den = den / den[0]
    num = num / den[0]
    n = len(den) - 1
    if n == 0:
        return (np.zeros((0, 0), order='F', dtype=float),
                np.zeros((0, 1), order='F', dtype=float),
                np.zeros((1, 0), order='F', dtype=float),
                np.array([[num[0]]], order='F', dtype=float))
    num_padded = np.zeros(n + 1)
    num_padded[n + 1 - len(num):] = num
    A = np.zeros((n, n), order='F', dtype=float)
    A[:-1, 1:] = np.eye(n - 1)
    A[-1, :] = -den[1:][::-1]
    B = np.zeros((n, 1), order='F', dtype=float)
    B[-1, 0] = 1.0
    C = np.zeros((1, n), order='F', dtype=float)
    d0 = num_padded[0]
    for i in range(n):
        C[0, n - 1 - i] = num_padded[i + 1] - d0 * den[i + 1]
    D = np.array([[d0]], order='F', dtype=float)
    return A, B, C, D

def ss_series(sys1, sys2):
    """Connect two state-space systems in series: sys2 * sys1."""
    A1, B1, C1, D1 = sys1
    A2, B2, C2, D2 = sys2
    n1, n2 = A1.shape[0], A2.shape[0]
    A = np.zeros((n1 + n2, n1 + n2), order='F', dtype=float)
    A[:n1, :n1] = A1
    A[n1:, :n1] = B2 @ C1
    A[n1:, n1:] = A2
    B = np.vstack([B1, B2 @ D1]).astype(float, order='F')
    C = np.hstack([D2 @ C1, C2]).astype(float, order='F')
    D = (D2 @ D1).astype(float, order='F')
    return A, B, C, D

def ss_feedback(sys, k=1.0):
    """Negative unity feedback: L / (1 + k*L)."""
    A, B, C, D = sys
    n = A.shape[0]
    inv_term = 1.0 / (1.0 + k * D[0, 0])
    Af = (A - k * inv_term * B @ C).astype(float, order='F')
    Bf = (inv_term * B).astype(float, order='F')
    Cf = (inv_term * C).astype(float, order='F')
    Df = (inv_term * D).astype(float, order='F')
    return Af, Bf, Cf, Df

def ss_sensitivity(sys, k=1.0):
    """Sensitivity function: 1 / (1 + k*L)."""
    A, B, C, D = sys
    n = A.shape[0]
    inv_term = 1.0 / (1.0 + k * D[0, 0])
    Af = (A - k * inv_term * B @ C).astype(float, order='F')
    Bf = (-k * inv_term * B).astype(float, order='F')
    Cf = (-inv_term * C).astype(float, order='F')
    Df = np.array([[inv_term]], order='F', dtype=float)
    return Af, Bf, Cf, Df

def simulate_step(sys, t, dt):
    """Simulate step response using slicot tf01md."""
    A, B, C, D = sys
    n = A.shape[0]
    if n == 0:
        return D[0, 0] * np.ones(len(t)), t
    Ad, Bd, Cd, Dd, info = ab04md('C', A.copy(order='F'), B.copy(order='F'),
                                   C.copy(order='F'), D.copy(order='F'),
                                   alpha=1.0, beta=2.0/dt)
    ny = len(t)
    u = np.ones((1, ny), order='F', dtype=float)
    x0 = np.zeros(n, dtype=float)
    y, x_final, info = tf01md(Ad, Bd, Cd, Dd, u, x0)
    return y.flatten(), t

# Example: G(s) = 6(s+5) / [(s+1)(s+2)(s+3)(s+4)]
num = [6, 30]
den = [1, 10, 35, 50, 24]

G = tf_to_ss(num, den)
print("Plant State-Space (A, B, C, D):")
print(f"A ({G[0].shape[0]}x{G[0].shape[1]}):\n{G[0]}")
print(f"B ({G[1].shape[0]}x{G[1].shape[1]}):\n{G[1]}")
print(f"C ({G[2].shape[0]}x{G[2].shape[1]}):\n{G[2]}")
print(f"D ({G[3].shape[0]}x{G[3].shape[1]}):\n{G[3]}")

### Plant Step Response Analysis

Visualize the plant's step response (including delay) to verify the model before controller tuning.

In [ ]:
poles = np.linalg.eigvals(G[0])
max_pole_freq = np.max(np.abs(poles)) if len(poles) > 0 else 1.0
dt = min(0.01, 1.0 / (20 * max_pole_freq))
t_plant = np.arange(0, 50, dt)

y_plant, t_plant_resp = simulate_step(G, t_plant, dt)

plt.figure(figsize=(7, 4))
plt.plot(t_plant_resp, y_plant, label='Plant Step Response')
plt.title('Plant Step Response')
plt.xlabel('Time (s)')
plt.ylabel('Output')
plt.grid()
plt.legend()
plt.show()

## 2. Specify PID Controller Parameters

Set your PID (or PI) controller parameters here.  
You can adjust these values and rerun the analysis cells below.

In [ ]:
Kp = 0.64362
Ki = 0.30314
Kd = 0.0

if Kd == 0:
    c_num = [Kp, Ki]
    c_den = [1, 0]
else:
    c_num = [Kd, Kp, Ki]
    c_den = [1, 0]

C = tf_to_ss(c_num, c_den)
print("Controller State-Space:")
print(f"A: {C[0]}")
print(f"B: {C[1]}")
print(f"C: {C[2]}")
print(f"D: {C[3]}")

## 3. Closed-Loop System and Step Responses

Analyze reference tracking and disturbance rejection.

In [ ]:
L = ss_series(G, C)
T = ss_feedback(L, 1.0)
S = ss_sensitivity(L, 1.0)

poles_T = np.linalg.eigvals(T[0])
max_freq_T = np.max(np.abs(poles_T)) if len(poles_T) > 0 else 1.0
dt_cl = min(0.01, 1.0 / (20 * max_freq_T))
t = np.arange(0, 50, dt_cl)

y_ref, t_ref = simulate_step(T, t, dt_cl)
y_dist, t_dist = simulate_step(S, t, dt_cl)

plt.figure(figsize=(10, 6))
plt.subplot(2, 1, 1)
plt.plot(t_ref, y_ref, label='Reference Tracking')
plt.title('Reference Tracking Step Response')
plt.ylabel('Output')
plt.legend()
plt.grid()

plt.subplot(2, 1, 2)
plt.plot(t_dist, y_dist, label='Disturbance Rejection (Sensitivity)')
plt.title('Disturbance Rejection Step Response')
plt.xlabel('Time (s)')
plt.ylabel('Output')
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

## 4. Stability Analysis and Margins

Check stability and compute gain/phase margins.

In [ ]:
A_L, B_L, C_L, D_L = L
n_L = A_L.shape[0]

poles_ol = np.linalg.eigvals(A_L) if n_L > 0 else np.array([])
poles_cl = np.linalg.eigvals(T[0]) if T[0].shape[0] > 0 else np.array([])

print("=" * 60)
print("Stability Analysis")
print("=" * 60)
print(f"\nOpen-Loop Poles: {poles_ol}")
print(f"\nClosed-Loop Poles: {poles_cl}")

is_stable = np.all(poles_cl.real < 0) if len(poles_cl) > 0 else True
print(f"\nClosed-Loop Stability: {'STABLE' if is_stable else 'UNSTABLE'}")

if not is_stable:
    unstable_poles = poles_cl[poles_cl.real >= 0]
    print(f"Unstable/Marginally Stable Poles: {unstable_poles}")

w = np.logspace(-3, 2, 500)
mag_ol = np.zeros(len(w))
phase_ol = np.zeros(len(w))

for i, freq in enumerate(w):
    s = 1j * freq
    if n_L > 0:
        g, rcond, evre, evim, hinvb, info = tb05ad(
            'N', 'G', A_L.copy(order='F'), B_L.copy(order='F'),
            C_L.copy(order='F'), s
        )
        H = g[0, 0] + D_L[0, 0]
    else:
        H = D_L[0, 0]
    mag_ol[i] = np.abs(H)
    phase_ol[i] = np.angle(H)

phase_deg = np.degrees(phase_ol)
phase_deg = np.unwrap(phase_deg * np.pi / 180) * 180 / np.pi

target_phase = -180
phase_error = np.abs(phase_deg - target_phase)
gm_idx = np.argmin(phase_error)
gm_freq = w[gm_idx]
gm_mag = mag_ol[gm_idx]
gm_db = 20 * np.log10(gm_mag) if gm_mag > 0 else -np.inf
gain_margin = -gm_db if gm_mag > 0 else 0

mag_db = 20 * np.log10(mag_ol + 1e-10)
mag_1_idx = np.argmin(np.abs(mag_db))
pm_freq = w[mag_1_idx]
pm_phase = phase_deg[mag_1_idx]
phase_margin = 180 + pm_phase

print(f"\n{'-' * 60}")
print("Frequency Domain Margins (Approximate)")
print(f"{'-' * 60}")
print(f"Gain Margin: {gain_margin:.2f} dB at {gm_freq:.4f} rad/s")
print(f"Phase Margin: {phase_margin:.2f} deg at {pm_freq:.4f} rad/s")

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

ax1.semilogx(w, mag_db)
ax1.axhline(0, color='r', linestyle='--', alpha=0.5, label='0 dB')
ax1.set_ylabel('Magnitude (dB)')
ax1.set_title('Open-Loop Bode Plot')
ax1.grid(True, which='both', alpha=0.3)
ax1.legend()

ax2.semilogx(w, phase_deg)
ax2.axhline(-180, color='r', linestyle='--', alpha=0.5, label='-180 deg')
ax2.set_xlabel('Frequency (rad/s)')
ax2.set_ylabel('Phase (degrees)')
ax2.grid(True, which='both', alpha=0.3)
ax2.legend()

plt.tight_layout()
plt.show()

## 5. Discretization for Embedded Deployment

Discretize the controller for digital implementation (e.g., embedded systems).

In [ ]:
Ts = 0.1

A_C, B_C, C_C, D_C = C
n_C = A_C.shape[0]

if n_C > 0:
    Ad_C, Bd_C, Cd_C, Dd_C, info = ab04md(
        'C', A_C.copy(order='F'), B_C.copy(order='F'),
        C_C.copy(order='F'), D_C.copy(order='F'),
        alpha=1.0, beta=2.0/Ts
    )
    C_d = (Ad_C, Bd_C, Cd_C, Dd_C)
else:
    C_d = C

print("=" * 60)
print("Discretization")
print("=" * 60)
print(f"\nSampling Period: {Ts} seconds")
print(f"\nContinuous Controller (State-Space):")
print(f"  A: {A_C}")
print(f"  B: {B_C.T}")
print(f"  C: {C_C}")
print(f"  D: {D_C}")
print(f"\nDiscrete-time Controller (Tustin):")
print(f"  A: {C_d[0]}")
print(f"  B: {C_d[1].T}")
print(f"  C: {C_d[2]}")
print(f"  D: {C_d[3]}")

## 6. Closed-Loop Analysis with Discrete Controller

Analyze the closed-loop system with the discretized controller.

In [ ]:
A_G, B_G, C_G, D_G = G
n_G = A_G.shape[0]

if n_G > 0:
    Ad_G, Bd_G, Cd_G, Dd_G, info = ab04md(
        'C', A_G.copy(order='F'), B_G.copy(order='F'),
        C_G.copy(order='F'), D_G.copy(order='F'),
        alpha=1.0, beta=2.0/Ts
    )
    G_d = (Ad_G, Bd_G, Cd_G, Dd_G)
else:
    G_d = G

L_d = ss_series(G_d, C_d)
T_d = ss_feedback(L_d, 1.0)

t_d = np.arange(0, 50, Ts)
ny_d = len(t_d)
u_d = np.ones((1, ny_d), order='F', dtype=float)
n_Td = T_d[0].shape[0]
x0_d = np.zeros(n_Td, dtype=float)

if n_Td > 0:
    y_d, x_final_d, info = tf01md(T_d[0], T_d[1], T_d[2], T_d[3], u_d, x0_d)
    y_resp = y_d.flatten()
else:
    y_resp = T_d[3][0, 0] * np.ones(ny_d)

y_cont, t_cont = simulate_step(T, np.arange(0, 50, dt_cl), dt_cl)

plt.figure(figsize=(10, 5))
plt.plot(t_cont, y_cont, 'b-', label='Continuous Closed-Loop', linewidth=2)
plt.plot(t_d, y_resp, 'ro-', label=f'Discrete Closed-Loop (Ts={Ts}s)', markersize=4, alpha=0.7)
plt.title('Continuous vs. Discrete Closed-Loop Step Response')
plt.xlabel('Time (s)')
plt.ylabel('Output')
plt.grid()
plt.legend()
plt.tight_layout()
plt.show()

print("\nDiscrete-time Plant (Tustin discretization):")
print(f"  A shape: {G_d[0].shape}")
poles_Td = np.linalg.eigvals(T_d[0]) if n_Td > 0 else np.array([])
print(f"\nDiscrete Closed-Loop Poles: {poles_Td}")
is_stable_d = np.all(np.abs(poles_Td) < 1) if len(poles_Td) > 0 else True
print(f"Discrete System Stability: {'STABLE' if is_stable_d else 'UNSTABLE'}")

## 7. Iterate: Adjust Plant or Controller and Re-run

You can modify the plant or controller parameters above and re-run the analysis cells as needed for iterative design and tuning.